In [5]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20
from bokeh.models import HoverTool, ColumnDataSource
output_notebook()

Loading BokehJS ...

## Loading Data

In [6]:
df = pd.read_csv("AllBirdsv4.csv")
df_test = pd.read_csv("Test Birds Location.csv")

### Pre-processing

#### All Birds file

In [7]:
# replace "?" characters
df["Y"] = df["Y"].map(lambda y: y.replace("?",""))
df["Y"] = df["Y"].astype('int64')
        
# Add a specific color for each specie according to Category20
birds_types = df['English_name'].sort_values().unique()
n_birds_types = len(birds_types)
birds_types_colors = pd.Series(Category20[n_birds_types], index = birds_types).rename('color')
df = df.join(birds_types_colors, on = "English_name")        
df['alpha'] = 1

# Convert date string to datetime
a = pd.to_datetime(df['Date'], errors = 'coerce', format = "%m/%d/%Y").dropna()
b = pd.to_datetime(df['Date'], errors = 'coerce', format = "%Y-%m-%d").dropna()
c = pd.to_datetime(df['Date'], errors = 'coerce', format = "%Y-%m-00").dropna()
df['T'] = pd.concat([a,b,c], axis = 0)

# Only keep valid date
df = df.loc[~df['T'].isna()].copy()

#### Test Birds file

In [8]:
# rename columns of X and Y
df_test.columns = ["ID", "X", "Y"]
df_test["name"] = "Test bird n° " + df_test['ID'].astype(str)
df_test["color"] = "red"

## Map visualization with Bokeh

In [16]:
source = ColumnDataSource(data=dict(
    x=df['X'],
    y=df['Y'],
    name=df["English_name"],
    color = df["color"]
))

source_test = ColumnDataSource(data=dict(
    x=df_test['X'],
    y=df_test['Y'],
    name=df_test["name"],
    color = df_test["color"]
))

source_dumping = ColumnDataSource(data=dict(
    x=[148],
    y=[159],
    name=[""]
))


TOOLS="crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

# put details on cursor
hover = HoverTool()
hover.tooltips = [
    ("Bird type", "@name"),
    ("Position", "(@x, @y)"),
]

p = figure(x_range=(0,200), y_range=(0,200), tools=TOOLS)

p.tools.append(hover)

# plot map
p.image_url( url=["map.jpg"],
             x=0, y=0, w=200, h=200, anchor="bottom_left")

# plot all birds
p.scatter(x = "x", y = "y", size=7, alpha=0.6, color="color", source = source)

# plot the 15 "test birds"
p.scatter(x = "x", y = "y", size=15, color="color", source = source_test, legend="Test birds")

# location of the dumping site
p.scatter(x = "x", y = "y", size=30, marker="circle_cross", fill_alpha=0.2, line_width=2, 
          source = source_dumping, legend="Dumping site")

show(p)